In [2]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Layer
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
class MF_layer(Layer):
    def __init__(self, user_num, item_num, latent_dim, use_bias=False, user_reg=1e-4, item_reg=1e-4,
                 user_bias_reg=1e-4, item_bias_reg=1e-4):
        """
        MF Layer
        :param user_num: user length
        :param item_num: item length
        :param latent_dim: latent number
        :param use_bias: whether using bias or not
        :param user_reg: regularization of user
        :param item_reg: regularization of item
        :param user_bias_reg: regularization of user bias
        :param item_bias_reg: regularization of item bias
        """
        super(MF_layer, self).__init__()
        self.user_num = user_num
        self.item_num = item_num
        self.latent_dim = latent_dim
        self.use_bias = use_bias
        self.user_reg = user_reg
        self.item_reg = item_reg
        self.user_bias_reg = user_bias_reg
        self.item_bias_reg = item_bias_reg

    def build(self, input_shape):
        self.p = self.add_weight(name='user_latent_matrix',
                                 shape=(self.user_num, self.latent_dim),
                                 initializer=tf.random_normal_initializer(),
                                 regularizer=l2(self.user_reg),
                                 trainable=True)
        self.q = self.add_weight(name='item_latent_matrix',
                                 shape=(self.item_num, self.latent_dim),
                                 initializer=tf.random_normal_initializer(),
                                 regularizer=l2(self.item_reg),
                                 trainable=True)
        self.user_bias = self.add_weight(name='user_bias',
                                         shape=(self.user_num, 1),
                                         initializer=tf.random_normal_initializer(),
                                         regularizer=l2(self.user_bias_reg),
                                         trainable=self.use_bias)
        self.item_bias = self.add_weight(name='item_bias',
                                         shape=(self.item_num, 1),
                                         initializer=tf.random_normal_initializer(),
                                         regularizer=l2(self.item_bias_reg),
                                         trainable=self.use_bias)
        
    def call(self, inputs, **kwargs):
        user_id, item_id, avg_score = inputs
        # MF
        latent_user = tf.nn.embedding_lookup(params=self.p, ids=user_id)
        latent_item = tf.nn.embedding_lookup(params=self.q, ids=item_id)
        outputs = tf.reduce_sum(tf.multiply(latent_user, latent_item), axis=1, keepdims=True)
        # MF-bias
        user_bias = tf.nn.embedding_lookup(params=self.user_bias, ids=user_id)
        item_bias = tf.nn.embedding_lookup(params=self.item_bias, ids=item_id)
        bias = tf.reshape((avg_score + user_bias + item_bias), shape=(-1, 1))
        # use bias
        outputs = bias + outputs if self.use_bias else outputs
        return outputs

In [28]:
latent_dim = 32
# use bias
use_bias = True

learning_rate = 0.001
batch_size = 512
epochs = 10
user_reg=1e-4
item_reg=1e-4

user_bias_reg=1e-4
item_bias_reg=1e-4

num_users = 30000
num_items = 30000
mf_layer = MF_layer(num_users, num_items, latent_dim, use_bias,
 user_reg, item_reg, user_bias_reg, item_bias_reg)


In [30]:
import pandas as pd
from tqdm import tqdm


def sparseFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}


def denseFeature(feat):
    """
    create dictionary for dense feature
    :param feat: dense feature name
    :return:
    """
    return {'feat': feat}


def create_explicit_ml_1m_dataset(file, latent_dim=4, test_size=0.2):
    """
    create the explicit dataset of movielens-1m
    We took the last 20% of each user sorted by timestamp as the test dataset
    Each of these samples contains UserId, MovieId, Rating, avg_score
    :param file: dataset path
    :param latent_dim: latent factor
    :param test_size: ratio of test dataset
    :return: user_num, item_num, train_df, test_df
    """
    data_df = pd.read_csv(file, sep=",", header=0,engine='python',
                     names=['UserId', 'MovieId', 'Rating', 'Timestamp'])
    data_df["UserId"].astype('int')
    data_df['avg_score'] = data_df.groupby(by='UserId')['Rating'].transform('mean')
    # feature columns
    user_num, item_num = data_df['UserId'].max() + 1, data_df['MovieId'].max() + 1
    feature_columns = [[denseFeature('avg_score')],
                       [sparseFeature('user_id', user_num, latent_dim),
                        sparseFeature('item_id', item_num, latent_dim)]]
    # split train dataset and test dataset
    watch_count = data_df.groupby(by='UserId')['MovieId'].agg('count')
    test_df = pd.concat([
        data_df[data_df.UserId == i].iloc[int(0.8 * watch_count[i]):] for i in tqdm(watch_count.index)], axis=0)
    test_df = test_df.reset_index()
    train_df = data_df.drop(labels=test_df['index'])
    train_df = train_df.drop(['Timestamp'], axis=1).sample(frac=1.).reset_index(drop=True)
    test_df = test_df.drop(['index', 'Timestamp'], axis=1).sample(frac=1.).reset_index(drop=True)

    train_X = [train_df['avg_score'].values, train_df[['UserId', 'MovieId']].values]
    train_y = train_df['Rating'].values.astype('int32')
    test_X = [test_df['avg_score'].values, test_df[['UserId', 'MovieId']].values]
    test_y = test_df['Rating'].values.astype('int32')
    return feature_columns, (train_X, train_y), (test_X, test_y)


In [32]:
test_size = 0.2

feature_columns, train, test = create_explicit_ml_1m_dataset("/Users/hui/Desktop/python/recommend/huichuanRecSys/spark_test/resources/ratings.csv",latent_dim, test_size)


100%|██████████| 29776/29776 [01:02<00:00, 475.01it/s]


In [42]:
mf_layer = MF_layer(30001, 1001, latent_dim, use_bias,
                                 user_reg, item_reg, user_bias_reg, item_bias_reg)

In [47]:
train_X, train_y = train
test_X, test_y = test

In [53]:
dense_feature_columns,sparse_feature_columns = feature_columns

In [54]:
dense_inputs = tf.keras.Input(shape=(len(dense_feature_columns),), dtype=tf.float32)
sparse_inputs = tf.keras.Input(shape=(len(sparse_feature_columns),), dtype=tf.int32)

In [57]:
user_id, item_id = sparse_inputs[:, 0], sparse_inputs[:, 1]
avg_score = dense_inputs
outputs = mf_layer([user_id, item_id, avg_score])

In [58]:
model = tf.keras.Model([dense_inputs,sparse_inputs], outputs)


In [59]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None,)]            0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [62]:
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
import numpy as np
model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate),
                metrics=['mse'])
# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=epochs,
    # callbacks=[checkpoint],
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test rmse: %f' % np.sqrt(model.evaluate(test_X, test_y)[1]))


Train on 830601 samples, validate on 92289 samples
Epoch 1/10


/Users/hui/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/hui/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


830601/830601 [==============================] - 19s 23us/sample - loss: 0.8099 - mse: 0.7785 - val_loss: 0.8164 - val_mse: 0.7821
Epoch 2/10
830601/830601 [==============================] - 18s 22us/sample - loss: 0.8091 - mse: 0.7711 - val_loss: 0.8177 - val_mse: 0.7797
Epoch 3/10
830601/830601 [==============================] - 19s 23us/sample - loss: 0.8086 - mse: 0.7680 - val_loss: 0.8179 - val_mse: 0.7778
Epoch 4/10
830601/830601 [==============================] - 20s 24us/sample - loss: 0.8082 - mse: 0.7661 - val_loss: 0.8179 - val_mse: 0.7771
Epoch 5/10
830601/830601 [==============================] - 18s 21us/sample - loss: 0.8077 - mse: 0.7647 - val_loss: 0.8178 - val_mse: 0.7764
Epoch 6/10
830601/830601 [==============================] - 18s 22us/sample - loss: 0.8072 - mse: 0.7639 - val_loss: 0.8176 - val_mse: 0.7756
Epoch 7/10
830601/830601 [==============================] - 18s 22us/sample - loss: 0.8068 - mse: 0.7629 - val_loss: 0.8178 - val_mse: 0.7752
Epoch 8/10
830601

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



test rmse: 0.905791
